In [30]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
import linear_model_stats as lms

Exercise 8

In [7]:
df = pd.read_csv('Auto.csv', na_values='?')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [12]:
# Get a msk of where horsepower is na
hp_na_msk = df['horsepower'].notnull()

# Set mpg as Y
Y = df.loc[hp_na_msk, 'mpg']

# Set horsepower as X
X = pd.DataFrame(df.loc[hp_na_msk, 'horsepower'])
X.dropna(inplace=True)
X.reset_index(inplace=True, drop=True)

In [14]:
reg = LinearRegression()
reg.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
# Get p-values for the coefficients
# Get standard error for the coefficients
# Used this: https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
from scipy import stats 

params = np.append(reg.intercept_,reg.coef_)
predictions = reg.predict(X)


newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))

MSE = (sum((Y-predictions)**2))/(len(newX) - len(newX.columns))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)


myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]

col_map = {0 : 'Intercept'}

for i, col in enumerate(X.columns):
    col_map[i + 1] = col

myDF3.index = myDF3.index.map(col_map)

print(myDF3)

            Coefficients  Standard Errors  t values  Probabilites
Intercept        39.9359            0.717    55.660           0.0
horsepower       -0.1578            0.006   -24.489           0.0


8.a.i

Based on the p-value there appears to be a relationship between mpg and horsepower

In [33]:
print('MPG mean:\t',Y.mean())
rse = lms.rse_arr(reg.predict(X), Y)
print('RSE:\t\t', rse)
print('Prcnt error:\t', rse/ Y.mean())
print('R^2:\t\t', reg.score(X, Y))

MPG mean:	 23.44591836734694
RSE:		 4.90575691954594
Prcnt error:	 0.2092371406691483
R^2:		 0.6059482578894348


8.b.ii

For every incrase one horsepower the mpg decreases by about .15.

Also, by looking above you can see that that on average the prediction will be off by about 21% (assuming the intercept and coefficent are known exactly).  The R^2 states that about 61% of the variability of mpg can be attributed to horsepower.

8.b.iii

Negative

8.b.iv

mpg = 39.9359 + (-0.1578 * 98) = 24.4715

*I don't currently have a way to detect confidence intervals.